## Importing Images Recursively

In [63]:
import glob
import numpy as np
import pandas as pd
from PIL import Image, ImageFilter
from skimage.color import rgb2gray
import cv2
import math

In [2]:
images = glob.glob("./Input/*.*")

## Adaptive Median Filter

In [3]:
def adaptive_median_filter(img, s=3, s_max=7):

    x, y = img.shape
    # Initialize result image
    result = np.zeros_like(img)

    # Traverse through image
    for i in range(0, x):
        for j in range(0, y):
            # Set current filter size to starting filter size
            s_cur = s
            # While current filter size is smaller or equal to maximum filter size
            while s_cur <= s_max:
                # Create new filter list
                filtr = []

                filter_edge = s_cur//2

                # Traverse through filter
                for u in range(s_cur):
                    for v in range(s_cur):
                        # Get current position
                        cur_x = (i + u - filter_edge)
                        cur_y = (j + v - filter_edge)

                        # Stay inside image boundaries
                        if((cur_x >= 0) and (cur_y >= 0) and (cur_x < x) and (cur_y < y)):
                            # Append value to filter list
                            filtr.append(img[cur_x, cur_y])

                            # Get value in center of filter region
                            if cur_x == i and cur_y == j:
                                z_xy = filtr[-1]

                # Convert filter list to numpy array
                filtr = np.asarray(filtr)
                # Get minimum value in filter region
                z_min = np.amin(filtr)
                # Get maximum value in filter region
                z_max = np.amax(filtr)
                # Get median value in filter region
                z_med = np.median(filtr)

                # If z_med is not an impulse: check next case. else: increase window size
                if z_min < z_med < z_max:
                    # If z_xy is not an impulse: output z_xy. else: output z_med
                    if z_min < z_xy < z_max:
                        result[i, j] = z_xy
                    else:
                        result[i, j] = z_med
                    # Break to exit while loop
                    break
                else:
                    s_cur += 2

            else:
                result[i, j] = z_med # Output median value if maximum window size has been surpassed

    return result

## Adaptive Local Noise Reduction Filter

In [6]:
var_g = np.var(input_images[0])
# print(var_g)

In [7]:
def adaptive_lnr_filter(img, var_g, s=3):

    x, y = img.shape
    # Initialize result image
    result = np.zeros_like(img)

    filter_edge = s//2

    # Traverse through image
    for i in range(0,x):
        for j in range(0,y):
            # Create new filter list
            filtr = []

            # Traverse through filter
            for u in range(s):
                for v in range(s):
                    # Get current position
                    cur_x = (i + u - filter_edge)
                    cur_y = (j + v - filter_edge)

                    # Stay inside image boundaries
                    if((cur_x >= 0) and (cur_y >= 0) and (cur_x < x) and (cur_y < y)):
                        # Append value to filter list
                        filtr.append(img[cur_x, cur_y])

            # Convert filter list to numpy array
            filtr = np.array(filtr)
            # Get local mean from filter
            mean_l = np.mean(filtr)
            # Get local variance from filter
            var_l = np.var(filtr)

            # If local variance is smaller than global variance, set ratio to 1
            if var_g <= var_l:
                r = var_g / var_l
            else:
                r = 1
            # Get the output value and round off to nearest integer
            result[i, j] = img[i, j] - (r * (img[i, j] - mean_l))

    return result

## Output Images

In [8]:
input_images = []
grayscale_images = []

output_images_med = []
output_images_lnr = []

for img in images:
    image_org = Image.open(img)
    image = np.array(image_org)
    
    input_images.append(image)
    
    grayscale_image = rgb2gray(image) #outputs a grayscaled image
    grayscale_images.append(grayscale_image)
    
    output = adaptive_median_filter(grayscale_image, 3, 7)
    output_images_med.append(output)
    
    output = adaptive_lnr_filter(grayscale_image , var_g)
    output_images_lnr.append(output)

C:\Users\Ahmenders\anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  del sys.path[0]


## Signal to Noise Ratio

In [9]:
def signaltonoise(a, axis=None, ddof=0):  #axis = None for single value output 
    a = np.asanyarray(a)
    m = a.mean(axis)
    sd = a.std(axis=axis, ddof=ddof)
    return np.where(sd == 0, 0, m/sd)

In [10]:
# FOR INPUT IMAGES
n = 1
for img in input_images:
    print(f'SNR of Input Image no. {n} :',signaltonoise(img,axis=None))
    n += 1

SNR of Input Image no. 1 : 2.128098666610694
SNR of Input Image no. 2 : 3.2918972157093718
SNR of Input Image no. 3 : 3.431399672412856
SNR of Input Image no. 4 : 3.059226953388642
SNR of Input Image no. 5 : 2.267202557563438


In [11]:
# FOR OUTPUT IMAGES OF ADAPTIVE MEDIAN FILTER
n = 1
for img in output_images_med:
    print(f'SNR of AMF Output Image no. {n} :',signaltonoise(img,axis=None))
    n += 1

SNR of AMF Output Image no. 1 : 2.130421814846192
SNR of AMF Output Image no. 2 : 3.299930378520058
SNR of AMF Output Image no. 3 : 3.4354787121674764
SNR of AMF Output Image no. 4 : 3.0648630953701708
SNR of AMF Output Image no. 5 : 2.2697847655527386


In [12]:
# FOR OUTPUT IMAGES OF ADAPTIVE LOCAL NOISE REDUCTION FILTER
n = 1
for img in output_images_lnr:
    print(f'SNR of ANLR Output Image no. {n} :',signaltonoise(img,axis=None))
    n += 1

SNR of ANLR Output Image no. 1 : 2.1357630879158727
SNR of ANLR Output Image no. 2 : 3.316781278421003
SNR of ANLR Output Image no. 3 : 3.4495881525384795
SNR of ANLR Output Image no. 4 : 3.0812519532271008
SNR of ANLR Output Image no. 5 : 2.271154329334771


## Entropy

In [13]:
from skimage.feature import greycomatrix
def _entropy(image):
    glcm = np.squeeze(greycomatrix(image, distances=[1], angles=[0], symmetric=True, normed=True))
    entropy = -np.sum(glcm*np.log2(glcm + (glcm==0)))
    return entropy

In [14]:
# FOR INPUT IMAGES
n = 1
for img in input_images:
    print(f'Entropy of Input Image no. {n} :',_entropy(img))
    n += 1

Entropy of Input Image no. 1 : 12.133920599166379
Entropy of Input Image no. 2 : 10.764561028414455
Entropy of Input Image no. 3 : 11.29525997931554
Entropy of Input Image no. 4 : 11.370063274497461
Entropy of Input Image no. 5 : 11.320869714157897


In [15]:
# FOR OUTPUT IMAGES OF ADAPTIVE MEDIAN FILTER
n = 1
for img in output_images_med:
    print(f'Entropy of AMF Output Image no. {n} :',_entropy(img))
    n += 1

Entropy of AMF Output Image no. 1 : 12.045095044793662
Entropy of AMF Output Image no. 2 : 10.6628580654782
Entropy of AMF Output Image no. 3 : 11.162306087796674
Entropy of AMF Output Image no. 4 : 11.249916612384313
Entropy of AMF Output Image no. 5 : 11.237540902388114


In [16]:
# FOR OUTPUT IMAGES OF ADAPTIVE LOCAL NOISE REDUCTION FILTER
n = 1
for img in output_images_lnr:
    print(f'Entropy of ANLR Output Image no. {n} :',_entropy(img))
    n += 1

Entropy of ANLR Output Image no. 1 : 11.848038800281103
Entropy of ANLR Output Image no. 2 : 10.499399085465814
Entropy of ANLR Output Image no. 3 : 10.90667950117403
Entropy of ANLR Output Image no. 4 : 10.946891362083917
Entropy of ANLR Output Image no. 5 : 11.062837457228149


## Brightness

In [66]:
def calculate_brightness(img):
    m , n = img.shape    

    pixel_sum = sum(sum(img))
    
    return pixel_sum/(m*n)

In [67]:
# FOR INPUT IMAGES
n = 1
for img in input_images:
    print(f'Brightness of Input Image no. {n} :',calculate_brightness(img))
    n += 1

Brightness of Input Image no. 1 : 0.4137313900291943
Brightness of Input Image no. 2 : 0.43989440833995147
Brightness of Input Image no. 3 : 0.433496269616671
Brightness of Input Image no. 4 : 0.4230937014127359
Brightness of Input Image no. 5 : 0.4396483260813637


In [72]:
# FOR OUTPUT IMAGES OF ADAPTIVE MEDIAN FILTER
n = 1
for img in output_images_med:
    print(f'Brightness of AMF Output Image no. {n} :',calculate_brightness(img))
    n += 1

Brightness of AMF Output Image no. 1 : 0.3979933110367893
Brightness of AMF Output Image no. 2 : 0.4498383686983367
Brightness of AMF Output Image no. 3 : 0.4263486985604188
Brightness of AMF Output Image no. 4 : 0.40410062527264795
Brightness of AMF Output Image no. 5 : 0.4366953389783112


In [73]:
# FOR OUTPUT IMAGES OF ADAPTIVE LOCAL NOISE REDUCTION FILTER
n = 1
for img in output_images_lnr:
    print(f'Brightness of ANLR Output Image no. {n} :',calculate_brightness(img))
    n += 1

Brightness of ANLR Output Image no. 1 : 0.43205333273677027
Brightness of ANLR Output Image no. 2 : 0.42595720405812015
Brightness of ANLR Output Image no. 3 : 0.4331942595720406
Brightness of ANLR Output Image no. 4 : 0.43289224952741023
Brightness of ANLR Output Image no. 5 : 0.40067784476683704


## Contrast

In [68]:
def calculate_contrast(img):
    m , n = img.shape
    brightness = calculate_brightness(img)

    
    image = img.flatten()
    
    total = 0
    for pixel in image:
        val = pow((pixel - brightness),2)
        total += val
    
    result = math.sqrt((1/(m*n))*total)
    
    return result

In [74]:
# FOR INPUT IMAGES
n = 1
for img in input_images:
    print(f'Contrast of Input Image no. {n} :',calculate_contrast(img))
    n += 1

Contrast of Input Image no. 1 : 137.15518032498625
Contrast of Input Image no. 2 : 177.63225902481636
Contrast of Input Image no. 3 : 142.5719482889602
Contrast of Input Image no. 4 : 137.51117430897094
Contrast of Input Image no. 5 : 110.44314360935195


In [75]:
# FOR OUTPUT IMAGES OF ADAPTIVE MEDIAN FILTER
n = 1
for img in output_images_med:
    print(f'Contrast of AMF Output Image no. {n} :',calculate_contrast(img))
    n += 1

Contrast of AMF Output Image no. 1 : 137.12800895474533
Contrast of AMF Output Image no. 2 : 177.5366284308088
Contrast of AMF Output Image no. 3 : 142.53716718365675
Contrast of AMF Output Image no. 4 : 137.40639860358706
Contrast of AMF Output Image no. 5 : 110.43140229603948


In [76]:
# FOR OUTPUT IMAGES OF ADAPTIVE LOCAL NOISE REDUCTION FILTER
n = 1
for img in output_images_lnr:
    print(f'Contrast of ANLR Output Image no. {n} :',calculate_contrast(img))
    n += 1

Contrast of ANLR Output Image no. 1 : 136.57620242590764
Contrast of ANLR Output Image no. 2 : 177.06968489890397
Contrast of ANLR Output Image no. 3 : 142.044798567778
Contrast of ANLR Output Image no. 4 : 136.87512718032767
Contrast of ANLR Output Image no. 5 : 109.97605957350747


## Saving Images in Output Folder

In [38]:
n = 1
for img in output_images_lnr:
    image = Image.fromarray(img)
    image.save(f"./Output/output{n}.png")
    n += 1